In [ ]:
url ='https://raw.githubusercontent.com/fbaptiste/python-deepdive/main/Part%202/Section%2004%20-%20Iterables%20and%20Iterators/cars.csv'

In [ ]:
import random
class Random:
  def __init__(self,n):
    self.n = n
    self.i = 0
  def __iter__(self):
    return self
  def __next__(self):
     if self.i >= self.n:
       raise StopIteration
     else:
       self.i += 1
       return random.randint(0,100)

if __name__ == '__main__':
  random.seed(0)
  l = list(Random(10))
  print(l)
  print()

[49, 97, 53, 5, 33, 65, 62, 51, 100, 38]



#PYDANTIC

In [ ]:
from pydantic import BaseModel
from typing import Optional

class Person(BaseModel):
    first_name: str
    last_name: str
    age: Optional[int]

p = Person(first_name='Isaac', last_name='Newton', age=84)

In [ ]:
from pydantic import ValidationError

try:
    Person(first_name='Isaac')
except ValidationError as ex:
    print(ex.json())

[
  {
    "loc": [
      "last_name"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  }
]


In [ ]:
p = Person(first_name='Isaac', last_name='Newton', age=84)

In [ ]:
from datetime import date

class Person(BaseModel):
    first_name: str = None
    last_name: str
    dob: date

data = {
    "first_name": "Isaac",
    "last_name": "Newton",
    "dob": date(1643, 1, 4)
}

p = Person.parse_obj(data)
p

Person(first_name='Isaac', last_name='Newton', dob=datetime.date(1643, 1, 4))

In [ ]:
json = '''
{
    "first_name": "Isaac",
    "last_name": "Newton",
    "dob": "1643-01-04"
}
'''
p = Person.parse_raw(json)
p

Person(first_name='Isaac', last_name='Newton', dob=datetime.date(1643, 1, 4))

#Field Aliases
One thing you may have noticed, is that the Pydantic model uses snake case for field names, since that's the convention for Python code - but in JSON we typically use camel case. Pydantic supports this!!

In [ ]:
from pydantic import Field

class Person(BaseModel):
    first_name: str = Field(alias='firstName', default=None)
    last_name: str = Field(alias='lastName')
    dob: date = None

try:
    Person.parse_obj(data)
except ValidationError as ex:
    print(ex.json())

[
  {
    "loc": [
      "lastName"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  }
]


In [ ]:
import uuid
from datetime import date, datetime, timedelta
import requests
from pydantic import BaseModel, conint,confloat

url = 'https://raw.githubusercontent.com/bugbytes-io/datasets/master/students_v1.json'

response = requests.get(url)
data = response.json()

#define Pydantic model class
class Student(BaseModel):
    id: uuid.UUID
    name: str
    date_of_birth: date
    GPA: confloat(gt=0, lt=130)
    course: str | None
    department: str
    fees_paid: bool

for student in data:
    # create Pydantic model object by unpacking key/val pairs from our JSON dict as arguments
    model = Student(**student)
    print(model)


id=UUID('d15782d9-3d8f-4624-a88b-c8e836569df8') name='Eric Travis' date_of_birth=datetime.date(1995, 5, 25) GPA=3.0 course='Computer Science' department='Science and Engineering' fees_paid=False
id=UUID('4c7b4c43-c863-4855-abc0-3657c078ce23') name='Mark Smith' date_of_birth=datetime.date(1996, 2, 10) GPA=2.5 course=None department='Science and Engineering' fees_paid=True
id=UUID('5cd9ad59-fcf1-462c-8863-282a9fb693e4') name='Marissa Barker' date_of_birth=datetime.date(1996, 10, 1) GPA=3.5 course='Biology' department='Life Sciences' fees_paid=False
id=UUID('48dda775-785d-41e3-b0dd-26a4a2f7722f') name='Justin Holden' date_of_birth=datetime.date(1994, 8, 22) GPA=3.23 course='Philosophy' department='Arts and Humanities' fees_paid=True
id=UUID('7ffe2ceb-562b-4edd-b74c-3741e1b08453') name='Michelle Thompson' date_of_birth=datetime.date(1995, 8, 5) GPA=3.9 course='Film Studies' department='Arts and Humanities' fees_paid=True


# Constrained Types

In [ ]:
import uuid
from datetime import date, datetime, timedelta
from pydantic import BaseModel, confloat, validator

class Student(BaseModel):
    id: uuid.UUID
    name: str
    date_of_birth: date
    GPA: confloat(ge=0, le=4)
    course: str | None
    department: str
    fees_paid: bool

    @validator('date_of_birth')
    def ensure_16_or_over(cls, value):
        sixteen_years_ago = datetime.now() - timedelta(days=365*16)

        # convert datetime object -> date
        sixteen_years_ago = sixteen_years_ago.date()

        # raise error if DOB is more recent than 16 years past.
        if value > sixteen_years_ago:
            raise ValueError("Too young to enrol, sorry!")
        return value

In [ ]:
for student in data:
    # create Pydantic model object by unpacking key/val pairs from our JSON dict as arguments
    model = Student(**student)
    print(model)


id=UUID('d15782d9-3d8f-4624-a88b-c8e836569df8') name='Eric Travis' date_of_birth=datetime.date(1995, 5, 25) GPA=3.0 course='Computer Science' department='Science and Engineering' fees_paid=False
id=UUID('4c7b4c43-c863-4855-abc0-3657c078ce23') name='Mark Smith' date_of_birth=datetime.date(1996, 2, 10) GPA=2.5 course=None department='Science and Engineering' fees_paid=True
id=UUID('5cd9ad59-fcf1-462c-8863-282a9fb693e4') name='Marissa Barker' date_of_birth=datetime.date(1996, 10, 1) GPA=3.5 course='Biology' department='Life Sciences' fees_paid=False
id=UUID('48dda775-785d-41e3-b0dd-26a4a2f7722f') name='Justin Holden' date_of_birth=datetime.date(1994, 8, 22) GPA=3.23 course='Philosophy' department='Arts and Humanities' fees_paid=True
id=UUID('7ffe2ceb-562b-4edd-b74c-3741e1b08453') name='Michelle Thompson' date_of_birth=datetime.date(1995, 8, 5) GPA=3.9 course='Film Studies' department='Arts and Humanities' fees_paid=True


#ENUM

In [ ]:
import uuid
from datetime import date, datetime, timedelta
from pydantic import BaseModel, confloat, validator
from enums import DepartmentEnum

class Student(BaseModel):
    id: uuid.UUID
    name: str
    date_of_birth: date
    GPA: confloat(ge=0, le=4)
    course: str | None
    department: DepartmentEnum
    fees_paid: bool

    @validator('date_of_birth')
    def ensure_16_or_over(cls, value):
        sixteen_years_ago = datetime.now() - timedelta(days=365*16)

        # convert datetime object -> date
        sixteen_years_ago = sixteen_years_ago.date()

        # raise error if DOB is more recent than 16 years past.
        if value > sixteen_years_ago:
            raise ValueError("Too young to enrol, sorry!")
        return value
for student in data:
    # create Pydantic model object by unpacking key/val pairs from our JSON dict as arguments
    model = Student(**student)
    print(model)


id=UUID('d15782d9-3d8f-4624-a88b-c8e836569df8') name='Eric Travis' date_of_birth=datetime.date(1995, 5, 25) GPA=3.0 course='Computer Science' department=<DepartmentEnum.SCIENCE_AND_ENGINEERING: 'Science and Engineering'> fees_paid=False
id=UUID('4c7b4c43-c863-4855-abc0-3657c078ce23') name='Mark Smith' date_of_birth=datetime.date(1996, 2, 10) GPA=2.5 course=None department=<DepartmentEnum.SCIENCE_AND_ENGINEERING: 'Science and Engineering'> fees_paid=True
id=UUID('5cd9ad59-fcf1-462c-8863-282a9fb693e4') name='Marissa Barker' date_of_birth=datetime.date(1996, 10, 1) GPA=3.5 course='Biology' department=<DepartmentEnum.LIFE_SCIENCES: 'Life Sciences'> fees_paid=False
id=UUID('48dda775-785d-41e3-b0dd-26a4a2f7722f') name='Justin Holden' date_of_birth=datetime.date(1994, 8, 22) GPA=3.23 course='Philosophy' department=<DepartmentEnum.ARTS_AND_HUMANITIES: 'Arts and Humanities'> fees_paid=True
id=UUID('7ffe2ceb-562b-4edd-b74c-3741e1b08453') name='Michelle Thompson' date_of_birth=datetime.date(1995,

# PYDANTIC - NESTED MODELS AND JSON SCHEMAS

Objectives
In this post, we will learn:

- How to create nested Pydantic models, and form parent-child relationships between model classes.
- How to use Literal types to constrain input values
- How to define default values for fields
- How to generate a JSON Schema from your Pydantic model definition
- How to use datamodel-code-generator to generate Pydantic models automatically from JSON Schema definitions
## Nested Pydantic model classes
- The data we worked with in the previous post was simple, flat data, as seen here. However, commonly in real-world apps, we need to deal with nested objects, each with their own schemas and fields.

- How do we use nested models in a pydantic context?

We have extended the dataset from the previous post to add the concept of nested modules to our student records. You can view the dataset, which we'll use in this post, here.

Each student has a set of modules that they are taking in the academic year, and these modules are their own objects with multiple fields.

In the previous post, we had the following pydantic model that described a hypothetical Student record in a college/university - note, other code (such as the DepartmentEnum ) can be found in the previous post:

In [ ]:
import uuid
import requests
import sys

sys.path.append('/content')
from typing import Literal
from datetime import date, datetime, timedelta

from pydantic import BaseModel, confloat, validator
from enums import DepartmentEnum
# from models import Student

# fetch the raw JSON data from Github
url = 'https://raw.githubusercontent.com/bugbytes-io/datasets/master/students_v2.json'
data = requests.get(url).json()



# Pydantic model to outline structure/types of Modules
class Module(BaseModel):
    id: int | uuid.UUID
    name: str
    professor: str
    credits: Literal[10,20]
    registration_code: str


class Student(BaseModel):
    id: uuid.UUID
    name: str
    date_of_birth: date
    GPA: confloat(ge=0, le=4)
    course: str | None
    department: DepartmentEnum
    fees_paid: bool
    modules: list[Module] = []

    @validator('modules')
    def validate_module_length(cls, value):
        if len(value) and len(value) != 3:
            raise ValueError('List of modules should have length 3')
        return value

    @validator('date_of_birth')
    def ensure_16_or_over(cls, value):
        sixteen_years_ago = datetime.now() - timedelta(days=365*16)

        # convert datetime object -> date
        sixteen_years_ago = sixteen_years_ago.date()

        # raise error if DOB is more recent than 16 years past.
        if value > sixteen_years_ago:
            raise ValueError("Too young to enrol, sorry!")
        return value

# for student in data:
#   model =Student(**student)
#   print(model)

print(Module.schema_json(indent=2))

{
  "title": "Module",
  "type": "object",
  "properties": {
    "id": {
      "title": "Id",
      "anyOf": [
        {
          "type": "integer"
        },
        {
          "type": "string",
          "format": "uuid"
        }
      ]
    },
    "name": {
      "title": "Name",
      "type": "string"
    },
    "professor": {
      "title": "Professor",
      "type": "string"
    },
    "credits": {
      "title": "Credits",
      "enum": [
        10,
        20
      ],
      "type": "integer"
    },
    "registration_code": {
      "title": "Registration Code",
      "type": "string"
    }
  },
  "required": [
    "id",
    "name",
    "professor",
    "credits",
    "registration_code"
  ]
}


In [ ]:
!pip install -q datamodel-code-generator

In [ ]:
sys.path.append('/content')
!datamodel-codegen --input jsonschema.json --output model2.py

In [ ]:
import uuid
import requests
import sys

sys.path.append('/content')
from typing import Literal
from datetime import date, datetime, timedelta

from pydantic import BaseModel, confloat, validator, Field
from enums import DepartmentEnum
# from models import Student

# fetch the raw JSON data from Github
url = 'https://raw.githubusercontent.com/bugbytes-io/datasets/master/students_v2.json'
data = requests.get(url).json()



# Pydantic model to outline structure/types of Modules
class Module(BaseModel):
    id: int | uuid.UUID
    name: str
    professor: str
    credits: Literal[10,20]
    registration_code: str


class Student(BaseModel):
    id: uuid.UUID
    name: str = Field(alias="name")
    date_of_birth: date = Field(default_factory=lambda: datetime.today().date())
    GPA: confloat(ge=0, le=4)
    course: str | None
    department: DepartmentEnum
    fees_paid: bool
    modules: list[Module] = Field(default=[], max_items=10)

    class Config:
        use_enum_values = True

    @validator('modules')
    def validate_module_length(cls, value):
        if len(value) and len(value) != 3:
            raise ValueError('List of modules should have length 3')
        return value

    @validator('date_of_birth')
    def ensure_16_or_over(cls, value):
        sixteen_years_ago = datetime.now() - timedelta(days=365*16)

        # convert datetime object -> date
        sixteen_years_ago = sixteen_years_ago.date()

        # raise error if DOB is more recent than 16 years past.
        if value > sixteen_years_ago:
            raise ValueError("Too young to enrol, sorry!")
        return value

for student in data:
  model =Student(**student)
  print(model)

# print(Module.schema_json(indent=2))

id=UUID('d15782d9-3d8f-4624-a88b-c8e836569df8') name='Eric Travis' date_of_birth=datetime.date(1995, 5, 25) GPA=3.0 course='Computer Science' department='Science and Engineering' fees_paid=False modules=[Module(id=1, name='Data Science and Machine Learning', professor='Prof. Susan Love', credits=20, registration_code='abc'), Module(id=UUID('e96e86a6-c4e0-4441-af43-0c22cc472e18'), name='Web Development', professor='Prof. James Herman', credits=20, registration_code='abc'), Module(id=3, name='Relational Databases and SQL', professor='Prof. Samantha Curtis', credits=10, registration_code='abc')]
id=UUID('4c7b4c43-c863-4855-abc0-3657c078ce23') name='Mark Smith' date_of_birth=datetime.date(1996, 2, 10) GPA=2.5 course=None department='Science and Engineering' fees_paid=True modules=[]
id=UUID('5cd9ad59-fcf1-462c-8863-282a9fb693e4') name='Marissa Barker' date_of_birth=datetime.date(1996, 10, 1) GPA=3.5 course='Biology' department='Life Sciences' fees_paid=False modules=[Module(id=55, name='Ge